In [3]:
!pip install tweet-preprocessor

In [1]:
!pip install nltk

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
#import preprocessor as p
import re
import warnings
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup
import string
import html
import requests
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('punkt')
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import os
import pickle
from sklearn.metrics import classification_report, accuracy_score

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/Liyuannnnn/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Liyuannnnn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/Liyuannnnn/nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/Liyuannnnn/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
def clean_text(_sentence):

    #removes emojis, pictographs, map symbols and flags
    emoji_pattern = re.compile(pattern = "["
          u"\U0001F600-\U0001F64F" 
          u"\U0001F300-\U0001F5FF" 
          u"\U0001F680-\U0001F6FF" 
          u"\U0001F1E0-\U0001F1FF" 
                            "]+", flags = re.UNICODE)
    _sentence = emoji_pattern.sub(r'',_sentence)

    #emoves tags and attributes
    _sentence = re.sub('(<[^>]*>)','',_sentence)

    #converts character code to string
    _sentence = html.unescape(_sentence)

    #removes urls
    _sentence = re.sub("(\w+:\/\/\S+)|(http[^\s]+)|(www.[^\s]+)",'',_sentence, flags=re.MULTILINE)
    
    #removes RT
    _sentence = re.sub("RT",'',_sentence)
    
    #converts the sentence to a lower case
    _sentence = _sentence.lower()

    #removes punctuation
    _sentence = re.sub(r'\@\w+|\#\w+','', _sentence)
    _sentence = _sentence.translate(str.maketrans('', '', (string.punctuation)))
    
    #checks for stop words
    stop_words = set(stopwords.words('english'))
    _sentence = [word for word in str(_sentence).split() if word not in stop_words]

    #joins the word and returns a sentence
    return " ".join(_sentence)

#function to preprocess tweets
def preprocess_tweets(tweets_df,column):

    #apply clean text in a vectorized format
    vfunc_clean_text = np.vectorize(clean_text)
    tweets_df[column] = vfunc_clean_text(tweets_df[column]) #create a new column that consists of preprocessed tweets
    
    return tweets_df

In [3]:
files = os.listdir('./tweet_data')

In [4]:
files

['tweet_putin_jun.csv',
 'tweet_putin_jul.csv',
 'tweet_ukraine_jun.csv',
 '.DS_Store',
 'tweet_ukrainewar_jun.csv',
 'tweet_russia_may.csv',
 'tweet_ukrainewar_jul.csv',
 'tweet_ukraine_jul.csv',
 'tweet_putin_may.csv',
 'tweet_ukrainewar_may.csv',
 'tweet_ukraine_may.csv',
 'tweet_russia_jun.csv',
 'tweet_russia_jul.csv',
 'tweet_russiawar_may.csv',
 'tweet_zelensky_may.csv',
 'tweet_russiawar_jun.csv',
 'tweet_russiawar_jul.csv',
 'tweet_zelensky_jun.csv',
 'tweet_zelensky_jul.csv']

## Reading Comments Into DataFrame

In [15]:
col = ["Unnamed:0", "ID", "Date", "Tweet"]
df_putin = pd.DataFrame(columns=col)
df_ukrainewar = pd.DataFrame(columns=col)
df_russia = pd.DataFrame(columns=col)
df_zelensky = pd.DataFrame(columns=col)
df_ukraine = pd.DataFrame(columns=col)
df_russiawar = pd.DataFrame(columns=col)

for file in files:
    if ".csv" in file:
        file_path = "./tweet_data/" + file
        df_temp = pd.read_csv(file_path)
        if "putin_" in file:
            df_putin = pd.concat([df_putin, df_temp])
        elif "ukrainewar_" in file:
            df_ukrainewar = pd.concat([df_ukrainewar, df_temp])
        elif "russia_" in file:
            df_russia = pd.concat([df_russia, df_temp])
        elif "zelensky_" in file:
            df_zelensky = pd.concat([df_zelensky, df_temp])
        elif "ukraine_" in file:
            df_ukraine = pd.concat([df_ukraine, df_temp])
        elif "russiawar_" in file:
            df_russiawar = pd.concat([df_russiawar, df_temp])

print("Number of tweets for #putin in May to July 2022: {}".format(len(df_putin)))
print("Number of tweets for #ukrainewar in May to July 2022: {}".format(len(df_ukrainewar)))
print("Number of tweets for #russia in May to July 2022: {}".format(len(df_russia)))
print("Number of tweets for #zelensky in May to July 2022: {}".format(len(df_zelensky)))
print("Number of tweets for #ukraine in May to July 2022: {}".format(len(df_ukraine)))
print("Number of tweets for #russiawar in May to July 2022: {}".format(len(df_russiawar)))

Number of tweets for #putin in May to July 2022: 506000
Number of tweets for #ukrainewar in May to July 2022: 506000
Number of tweets for #russia in May to July 2022: 506000
Number of tweets for #zelensky in May to July 2022: 506000
Number of tweets for #ukraine in May to July 2022: 506000
Number of tweets for #russiawar in May to July 2022: 506000


In [17]:
data_df_putin = preprocess_tweets(df_putin,'Tweet')
data_df_ukrainewar = preprocess_tweets(df_ukrainewar,'Tweet')
data_df_russia = preprocess_tweets(df_russia,'Tweet')
data_df_zelensky = preprocess_tweets(df_zelensky,'Tweet')
data_df_ukraine = preprocess_tweets(df_ukraine,'Tweet')
data_df_russiawar = preprocess_tweets(df_russiawar,'Tweet')

## Determine Labels Using NLTK Vader Toolkit

In [24]:
def Vader_process(data_df):
    sid = SentimentIntensityAnalyzer()
    res = [*data_df['Tweet'].apply(sid.polarity_scores)]
    #print(res[:3])

    sentiment_df = pd.DataFrame.from_records(res)
    data_df = pd.concat([data_df.reset_index(drop=True), sentiment_df.reset_index(drop=True)], axis=1, join="inner")
    #print(data_df.head())
    
    conditions = [
    (data_df['compound'] <= -0.5),
    (data_df['compound'] > -0.5) & (data_df['compound'] < 0.5),
    (data_df['compound'] >= 0.5)
    ]

    values = [0, -1, 1]
    data_df['label'] = np.select(conditions, values)

    #print(data_df.head())
    
    X_df = data_df[data_df.label != -1]
    X_df = X_df[['Tweet','label']]
    #print(X_df.head())
    
    return X_df
    

In [25]:
X_putin = Vader_process(data_df_putin)
X_ukrainewar = Vader_process(data_df_ukrainewar)
X_russia = Vader_process(data_df_russia)
X_zelensky = Vader_process(data_df_zelensky)
X_ukraine = Vader_process(data_df_ukraine)
X_russiawar = Vader_process(data_df_russiawar)


In [26]:
X_putin.groupby('label').count()

,Tweet
label,
0,139857
1,74094


## Train Logistic Regression Model

In [27]:
df = pd.read_csv('sentiment_analysis.csv')
df = df.drop(['ID'], axis=1)
df.head()

,text,label
0,Josh Jenkins is looking forward to TAB Breeder...,1
1,RT @MianUsmanJaved: Congratulations Pakistan o...,1
2,"RT @PEPalerts: This September, @YESmag is taki...",1
3,"RT @david_gaibis: Newly painted walls, thanks ...",1
4,RT @CedricFeschotte: Excited to announce: as o...,1


In [29]:
df = preprocess_tweets(df,"text")

In [30]:
df

,text,label
0,josh jenkins looking forward tab breeders crow...,1
1,congratulations pakistan becoming world odds,1
2,september taking maine mendoza’s surprise than...,1
3,newly painted walls thanks million custodial p...,1
4,excited announce july 2017 feschotte lab reloc...,1
...,...,...
550386,cant stop watching hermm,0
550387,poor old tom odell doesnt look like would know...,0
550388,smashed 7 ants awesome gamehjfjfi,1
550389,morning girls wonderful,1


In [31]:
X = df.loc[:, ['text']]
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

Logistic Regression

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
bag = count.fit_transform(X_train['text'].tolist())

In [33]:
import numpy as np

from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True,
                         norm='l2',
                         smooth_idf=True)

np.set_printoptions(precision=2)

In [34]:
from collections import Counter

vocab = Counter()
for twit in X_train.text:
    for word in twit.split(' '):
        vocab[word] += 1

vocab.most_common(20)

[('love', 42385),
 ('happy', 40825),
 ('great', 23262),
 ('good', 21919),
 ('im', 20543),
 ('best', 20086),
 ('amazing', 19574),
 ('day', 18533),
 ('birthday', 17562),
 ('one', 13723),
 ('thank', 13584),
 ('found', 13340),
 ('dont', 12588),
 ('like', 12383),
 ('beautiful', 12258),
 ('transponder', 11427),
 ('much', 11133),
 ('get', 10924),
 ('today', 10857),
 ('cant', 10325)]

In [35]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [36]:
import math

def plot_distribution(vocabulary):

    hist, edges = np.histogram(list(map(lambda x:math.log(x[1]),vocabulary.most_common())), density=True, bins=500)

    p = figure(tools="pan,wheel_zoom,reset,save",
               toolbar_location="above",
               title="Word distribution accross all twits")
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555", )
    show(p)

plot_distribution(vocab)

In [37]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

vocab_reduced = Counter()
for w, c in vocab.items():
    if not w in stop:
        vocab_reduced[w]=c

vocab_reduced.most_common(20)

plot_distribution(vocab_reduced)

In [38]:
from nltk.stem import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):
    return text.split()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [39]:
# Best parameters
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)


param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__tokenizer': [tokenizer],
               'clf__penalty': ['l2'],
               'clf__C': [10.0],
              'clf__solver': ['liblinear'],
              'clf__max_iter': [1000]}
]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1)

In [40]:
# Note: This process took 25 hours
gs_lr_tfidf.fit(X_train.values.ravel(), y_train.values.ravel())

Fitting 5 folds for each of 1 candidates, totalling 5 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0))]),
             param_grid=[{'clf__C': [10.0], 'clf__max_iter': [1000],
                          'clf__penalty': ['l2'], 'clf__solver': ['liblinear'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__tokenizer': [<function tokenizer at 0x7f87b2406820>]}],
             scoring='accuracy', verbose=1)

In [41]:
#print('Best parameter set: ' + str(gs_lr_tfidf.best_params_))
print('Best accuracy: %.3f' % gs_lr_tfidf.best_score_)

Best accuracy: 0.974


In [42]:
clf = gs_lr_tfidf.best_estimator_
#clf.fit(X_train.values, y_train)
clf.fit(X_test['text'].tolist(), y_test)
print('Accuracy in test: %.3f' % clf.score(X_test['text'].tolist(), y_test))

Accuracy in test: 0.992


## Predict Labels with Logistic Regression Model

In [47]:
def predict(X_df):
    X_df['Predicted Label'] = clf.predict(X_df['Tweet'])
    #print(X_df.head())
    print(classification_report(X_df['label'],X_df['Predicted Label']))

In [49]:
print("Result for #putin from May to July 2022:")
predict(X_putin)
print("Result for #ukrainewar from May to July 2022:")
predict(X_ukrainewar)
print("Result for #russia from May to July 2022:")
predict(X_russia)
print("Result for #zelensky from May to July 2022:")
predict(X_zelensky)
print("Result for #ukraine from May to July 2022:")
predict(X_ukraine)
print("Result for #russiawar from May to July 2022:")
predict(X_russiawar)

Result for #putin from May to July 2022:
              precision    recall  f1-score   support

           0       0.76      0.98      0.86    139857
           1       0.93      0.40      0.56     74094

    accuracy                           0.78    213951
   macro avg       0.84      0.69      0.71    213951
weighted avg       0.82      0.78      0.75    213951

Result for #ukrainewar from May to July 2022:
              precision    recall  f1-score   support

           0       0.96      0.97      0.97    307985
           1       0.63      0.53      0.58     24104

    accuracy                           0.94    332089
   macro avg       0.79      0.75      0.77    332089
weighted avg       0.94      0.94      0.94    332089

Result for #russia from May to July 2022:
              precision    recall  f1-score   support

           0       0.76      0.99      0.86    143908
           1       0.93      0.35      0.51     69823

    accuracy                           0.78    213731